In [ ]:
#default_exp checkpoint

In [ ]:
#hide_output
%load_ext autoreload
%autoreload 2

# Checkpointing
> Saving and restoring graphs, reachability labels, evaluation results, etc.

In [ ]:
#export
# creating graphs in Python
import networkx as nx
# checking for existence of paths, and manipulating paths
from pathlib import Path, PurePath
# data analysis and manipulation
import pandas as pd
# reachability labels
from git_commit_graph_ext.labelling.levels import find_levels
from git_commit_graph_ext.labelling.dfs_intervals import find_dfs_intervals, find_dfs_intervals_extra

## Functions for computing reachability labels, and saving them and graph to a file, etc.

In [ ]:
# imports for example graphs
import git_commit_graph_ext.example_graphs as graphs

In [ ]:
# before tests (test fixture)

# example graph
example_graph = graphs.commit_graph_Stolee()
example_graph_name = 'commit_graph_Stolee'

# check that it worked
print("the example commit graph '{}' from Stolee blog posts has {:d} nodes and {:d} edges".
      format(example_graph_name, example_graph.number_of_nodes(), example_graph.number_of_edges()))

the example commit graph 'commit_graph_Stolee' from Stolee blog posts has 23 nodes and 31 edges


### Functions for saving graph structure as `DataFrame` (storing edge list) to a file, and restoring it

Helper function to get basename for saving graph and/or graph data to a file (as a dataset).

In [ ]:
#export
def _savefile_name(graph_name, out_dir='datasets', kind='df_edgelist', file_format='csv.gz'):
    """Create filename for storing graph structure and other graph data

    This is a helper function used, among others, in ...

    Examples:
    ---------
    >>>> _savefile_name('example_graph')
    Path('datasets/example_graph.df_edgelist.csv.gz')

    Parameters
    ----------
    graph_name : str
        Name of the graph (`<graph>.name` can be used).

    out_dir : str
        Directory where saved commit graph data would be stored.
        Defaults to "datasets".

    kind : str
        What type of data is stored in a file, and in what representation.
        The default value is 'df_edgelist', used to store graph structure in
        the edge list format in a `pandas.DataFrame`.

    file_format : str
        Format of a file, for example how the `DataFrame` is saved.
        Defaults to 'csv.gz' (gzip-compressed Comma Separated Values).

    Returns
    -------
    Path
        Path to the file storing the graph structure or graph data in
        the appropriate representation and appropriate file format.
    """
    # The `out_dir` should not be None
    if out_dir is None:
        out_dir = "."

    # compose the basename of the pathname
    filename = graph_name
    # TODO: there would special case for saving to HDF5 files, which can
    # store multiple data in a single file, so there would be no need
    # to add <kind> to basename of such output file
    if kind is not None and kind != '':
        filename += '.' + kind
    if file_format is not None and file_format != '':
        filename += '.' + file_format
    # generate the name of the output file, as `pathlib.Path` object
    return Path(out_dir) / filename


def _out_basename(graph_name, out_dir='datasets'):
    return _savefile_name(graph_name, out_dir=out_dir, kind=None, file_format=None)



<u>Test</u> that examples from the docstring works:

In [ ]:
assert _savefile_name('example_graph') == Path('datasets/example_graph.df_edgelist.csv.gz')
assert _savefile_name('example_graph', kind='adjlist', file_format='txt') == Path('datasets/example_graph.adjlist.txt')
assert _savefile_name('example_graph', out_dir='data', kind='adjlist', file_format='txt') == Path('data/example_graph.adjlist.txt')

Save graph to a `DataFrame` (saving containing edgelist information), and restore it.

In [ ]:
#export
def graph_to_dataframe(graph):
    return nx.to_pandas_edgelist(graph)


def dataframe_to_graph(df):
    return nx.from_pandas_edgelist(df, create_using=nx.DiGraph)



<u>Test</u> that saving to the `DataFrame` and restoring from it works correctly,... up to nodes that are not connected -- those cannot be stored using only edge list data, but with addition of node list it should be possible to restore graph exactly:

In [ ]:
example_graph_df = graph_to_dataframe(example_graph)
assert len(example_graph_df.index) == example_graph.number_of_edges()
print('ok - number of rows in dataframe with edgelist {} matches number of edges {}'.
      format(len(example_graph_df.index), example_graph.number_of_edges()))
restored_graph = dataframe_to_graph(example_graph_df)
assert restored_graph.number_of_edges() == example_graph.number_of_edges()
print('ok - number of edges {} in restored graph matches number of edges {} in the original'.
      format(restored_graph.number_of_edges(), example_graph.number_of_edges()))
assert restored_graph.edges == example_graph.edges
print('ok - all edges from the original graph got restored')
assert set(restored_graph).issubset(example_graph)
print('ok - all restored nodes are in the original graph')

ok - number of rows in dataframe with edgelist 31 matches number of edges 31
ok - number of edges 31 in restored graph matches number of edges 31 in the original
ok - all edges from the original graph got restored
ok - all restored nodes are in the original graph


Save graph to a file (via `DataFrame` containing edgelist information), and restore it

In [ ]:
#export
def guess_format(filename):
    suffixes = PurePath(filename).suffixes
    file_format = suffixes[-1]
    if file_format == '.gz' or file_format == '.txt':
        file_format = suffixes[-2] + file_format
    return file_format[1:]


def save_df_to_file(df, filename, output_format='csv.gz'):
    if output_format is None:
        output_format = guess_format(filename)
    if output_format == 'csv' or output_format == 'csv.gz':
        df.to_csv(filename)
    else:
        raise NotImplementedError("Writing to '{}' format is not supported".format(output_format))


def save_graph_df(df, graph_name, datasets_dir='datasets', output_format='csv.gz', overwrite=False):
    filename = _savefile_name(graph_name, out_dir=datasets_dir,
                              kind='df_edgelist', file_format=output_format)
    print('-> filename:', filename)
    if not overwrite and Path(filename).is_file():
        return
    save_df_to_file(df, filename, output_format=output_format)


def save_graph(graph, graph_name=None, datasets_dir='datasets', output_format='csv.gz', overwrite=False):
    df = graph_to_dataframe(graph)
    # if `graph_name` is not given, check the `name` attribute of the `graph`
    if graph_name is None:
        # NOTE: "'name' in graph" checks if there is node named 'name' in the graph
        if hasattr(graph, 'name'):
            graph_name = graph.name
        else:
            raise RuntimeError("Neither 'graph_name' parameter given, nor 'graph' has 'name' attribute")

    print('-> graph_name:', graph_name)
    save_graph_df(df, graph_name,
                  datasets_dir=datasets_dir, output_format=output_format, overwrite=overwrite)


def load_df_from_file(filename, input_format='csv.gz'):
    if input_format is None:
        input_format = guess_format(filename)
    if input_format == 'csv' or input_format == 'csv.gz':
        return pd.read_csv(filename, index_col=0)
    else:
        raise NotImplementedError("Reading from '{}' format is not supported".format(input_format))


def load_graph_df(graph_name, datasets_dir='datasets', input_format='csv.gz'):
    filename = _savefile_name(graph_name, out_dir=datasets_dir,
                              kind='df_edgelist', file_format=input_format)
    print('<- filename:', filename)
    return load_df_from_file(filename, input_format=input_format)



<u>Test</u> guessing file format from file name

In [ ]:
assert guess_format('datasets/hellogitworld-commit_graph.df_edgelist.csv.gz') == 'csv.gz'
assert guess_format('datasets/hellogitworld-commit_graph.df_edgelist.csv') == 'csv'
assert guess_format('datasets/hellogitworld-commit_graph.adjlist.txt') == 'adjlist.txt'

<u>Test</u> saving graph structure (via `DataFrame`) to a file, and restoring / reading such `DataFrame`.

In [ ]:
print('graph.name = {}'.format(example_graph.name))
print('testing save_graph()')
save_graph(example_graph)
print('testing save_graph_df()')
save_graph_df(example_graph_df, graph_name=example_graph_name)
print('there should be appropriately named file in the list below:')
["{name:<50} {size:>7}".format(name=p.name, size=p.stat().st_size)
 for p in Path("datasets").glob(example_graph_name+"*")]

graph.name = 
testing save_graph()
-> graph_name: 
-> filename: datasets\.df_edgelist.csv.gz
testing save_graph_df()
-> filename: datasets\commit_graph_Stolee.df_edgelist.csv.gz
there should be appropriately named file in the list below:


['commit_graph_Stolee.df_edgelist.csv.gz                 231']

In [ ]:
print('restoring graph named "{}"'.format(example_graph_name))
df = load_graph_df(example_graph_name)
assert example_graph_df.equals(df)
print('ok - dataframe and restored dataframe are equal')

restoring graph named "commit_graph_Stolee"
<- filename: datasets\commit_graph_Stolee.df_edgelist.csv.gz
ok - dataframe and restored dataframe are equal


### Functions for computing reachability levels, saving them to a file (as `DataFrame`), and restoring them

Compute levels and min-post intervals for a graph, and store them as attributes of the graph object

In [ ]:
#export
def compute_reachability_labels(graph, recompute=False):
    if recompute or not hasattr(graph, 'lvl'):
        graph.lvl = find_levels(graph)
    if recompute or not hasattr(graph, 'mpi_ext'):
        graph.mpi_ext = find_dfs_intervals_extra(graph)
    return graph



<u>Test</u> computing reachability labels and saving them as attributes of the graph object

In [ ]:
print('compute reachability labels for {} graph'.format(example_graph_name))
compute_reachability_labels(example_graph)

print('we should see {} and {} among dict-values public attributes'.format('lvl', 'mpi_ext'))
for (attr, val) in example_graph.__dict__.items():
    if isinstance(val, dict) and not attr.startswith('_'):
        print('- {:s}'.format(attr))
assert hasattr(example_graph, 'lvl')
assert hasattr(example_graph, 'mpi_ext')
print('ok - graph has both "{}" and "{}" attributes'.format('lvl', 'mpi_ext'))

print('reachability labels should be computed for all nodes')
assert set(example_graph.lvl.keys()) == set(example_graph.nodes)
assert set(example_graph.mpi_ext.keys()) == set(example_graph.nodes)
print('ok - both lvl and mpi_ext keys are all {} graph nodes'.format(len(example_graph.nodes)))

compute reachability labels for commit_graph_Stolee graph
we should see lvl and mpi_ext among dict-values public attributes
- graph
- pos
- lvl
- mpi_ext
ok - graph has both "lvl" and "mpi_ext" attributes
reachability labels should be computed for all nodes
ok - both lvl and mpi_ext keys are all 23 graph nodes


Store reachability labels and per-node information in a `DataFrame`

In [ ]:
#export
def graph_data_to_dataframe(graph, append_to=None):
    compute_reachability_labels(graph)

    # create the DataFrame and name its index
    df = pd.DataFrame.from_dict(graph.mpi_ext, orient='index', columns=['f_min', 'min', 'post'])
    df.index.name = 'node'
    # add other reachability labels
    df['level'] = pd.Series(graph.lvl)
    # add and compute other data
    df['in degree'] = pd.Series(dict(graph.in_degree()))
    df['out degree'] = pd.Series(dict(graph.out_degree()))
    df['degree'] = df['in degree'] + df['out degree']

    # append if needed
    if append_to:
        df = pd.concat([append_to, df], axis=1, join='inner')

    return df



<u>Test</u> computing all per-node data for a graph and storing them in `DataFrame`

In [ ]:
df = graph_data_to_dataframe(example_graph)

print('check that the dataframe has all the columns')
assert set(df.columns) == set(['level', 'f_min', 'min', 'post', 'in degree', 'out degree', 'degree'])
print('- columns: {}'.format(df.columns.tolist()))

print('check that the dataframe has all the rows')
assert set(df.index) == set(example_graph.nodes)
print('- rows:    {}...'.format(list(example_graph.nodes)[0:5]))

df.head()

check that the dataframe has all the columns
- columns: ['f_min', 'min', 'post', 'level', 'in degree', 'out degree', 'degree']
check that the dataframe has all the rows
- rows:    ['A', 'a7', 'a5', 'a4', 'a3']...


,f_min,min,post,level,in degree,out degree,degree
node,,,,,,,
b0,1,1,1,0,3,0,3
a1,1,1,2,1,2,1,3
a2,1,1,3,2,1,1,2
a3,1,1,4,3,2,1,3
a4,1,1,5,4,1,1,2


Compute and save graph structure, its reachability labels and other per-node info to a file, and restore it.  
Don't redo calculations that can be retrieved from a file.

In [ ]:
#export
def compute_cached_df(code, filename, file_format=None, dont_save=False):
    """Compute `DataFrame`, or retrieve it from a given file if it exists

    Parameters:
    -----------
    code : callable
        Code to call if the `filename` file does not exist.  It should be
        a parameter-less function that returns a `DataFrame`.

    filename : str | Path
        Name of the file that, if exists, stores the `DataFrame`

    file_format : str
        Format of a file, for example how the `DataFrame` is saved.
        Defaults to None (guess from the file name).

    Returns:
    --------
    DataFrame
        Recomputed or loaded from provided file DataFrame.
    """
    if Path(filename).is_file():
        return load_df_from_file(filename, input_format=file_format)
    else:
        df = code()
        if not dont_save:
            save_df_to_file(df, filename, output_format=file_format)
        return df



<u>Test</u> that the code runs if the file does not exist.

In [ ]:
compute_cached_df(lambda: print("the provided code ran"), 'this_file_does_not_exist.xxx', dont_save=True)

the provided code ran


In [ ]:
test_df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
assert compute_cached_df(lambda: test_df, 'this_file_does_not_exist.xxx', dont_save=True)\
                         .equals(test_df)

<u>Test</u> that the code do not run if the file exists, and that we get what we saved

In [ ]:
expect_df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
df_filename = 'datasets/example_dataframe.csv.gz'
save_df_to_file(expect_df, df_filename)
actual_df = compute_cached_df(lambda: print("THIS SHOULD NOT RUN"), df_filename)
assert actual_df.equals(expect_df)

<u>Test</u> that `compute_cached_df` saves results to a file

In [ ]:
_counter = 0

def example_dataframe():
    global _counter
    _counter = _counter + 1
    return pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

expect_df = example_dataframe()
_counter = 0

df_filename = 'datasets/example_dataframe.csv.gz'
# Path(df_filename).unlink(missing_ok=True)
if not Path(df_filename).exists():
    Path(df_filename).unlink()

# first time it should perform computations and save, second time get from file
actual_df = compute_cached_df(example_dataframe, df_filename)
actual_df = compute_cached_df(example_dataframe, df_filename)

print('dataframe:\n{}\n'.format(actual_df))

# tests
assert actual_df.equals(expect_df)
print('ok - extracted dataframe matches the one being saved')
assert Path(df_filename).exists() and Path(df_filename).is_file()
print('ok - there exist file "{}"'.format(df_filename))
assert _counter == 1
print('ok - the `example_dataframe()` was called only 1 time (actual: {} time(s))'.format(_counter))

dataframe:
   col1  col2
0     1     3
1     2     4

ok - extracted dataframe matches the one being saved
ok - there exist file "datasets/example_dataframe.csv.gz"
ok - the `example_dataframe()` was called only 1 time (actual: 1 time(s))


Compute `DataFrame` with graph structure (`compute_cached_graph_df()`) and reachability labels (`compute_cached_reachability_labels_df()`) and save it to a file, or retrieve it from a file if it exists.

In [ ]:
#export
def compute_cached_graph_df(graph_generator, graph_name,
                            datasets_dir='datasets', file_format='csv.gz'):
    filename = _savefile_name(graph_name, out_dir=datasets_dir,
                              kind='df_edgelist', file_format=file_format)
    return compute_cached_df(lambda: graph_to_dataframe(graph_generator()), filename)


def compute_cached_reachability_labels_df(graph, graph_name=None, append_to=None,
                                          datasets_dir='datasets', file_format='csv.gz'):
    # if `graph_name` is not given, check the `name` attribute of the `graph`
    if graph_name is None:
        # NOTE: "'name' in graph" checks if there is node named 'name' in the graph
        if hasattr(graph, 'name'):
            graph_name = graph.name
        else:
            raise RuntimeError("Neither 'graph_name' parameter given, nor 'graph' has 'name' attribute")
    # generate filename to save to
    filename = _savefile_name(graph_name, out_dir=datasets_dir,
                              kind='df_nodedata', file_format=file_format)
    # return computer or retrieved dataframe
    return compute_cached_df(lambda: graph_data_to_dataframe(graph, append_to=append_to), filename)



<u>Test</u> `compute_cached_graph_df()` and `compute_cached_reachability_labels_df()`

In [ ]:
graph_df = compute_cached_graph_df(graphs.commit_graph_Stolee, 'commit_graph_Stolee-test')
assert graph_df.equals(graph_to_dataframe(graphs.commit_graph_Stolee()))

graph_data_df = compute_cached_reachability_labels_df(example_graph, example_graph_name)
assert graph_data_df.equals(graph_data_to_dataframe(example_graph))

To generate graph out of dataframe, use `dataframe_to_graph()`, defined earlier.

To retrieve reachability labels from the dataframe, and store them in graph attributes, use `dataframe_to_reachability_labels()`.

In [ ]:
#export
def dataframe_to_reachability_labels(df, graph, recompute=False):
    #print('..dataframe_to_reachability_labels({}, {!r}, {})'.format(type(df), graph, recompute))
    if recompute or not hasattr(graph, 'lvl'):
        #print("...recovering 'level' data")
        graph.lvl = df['level'].to_dict()
    if recompute or not hasattr(graph, 'mpi_ext'):
        #print("...recovering 'mpi_ext' data")
        graph.mpi_ext = df[['f_min', 'min', 'post']].to_dict(orient='index')
    return graph

In [ ]:
# check the assumptions of the code
assert df[['f_min', 'min', 'post']].to_dict(orient='index') == example_graph.mpi_ext
assert df['level'].to_dict() == example_graph.lvl

Generate graph, or retrieve it from saved `DataFrame` with edge list data.

Generate reachability labels for the graph, or retrieve them from saved `DataFrame` with per-node data, adding missing nodes if necessary

In [ ]:
#export
def compute_cached_graph(graph_generator, graph_name, datasets_dir='datasets', file_format='csv.gz'):
    filename = _savefile_name(graph_name, out_dir=datasets_dir,
                              kind='df_edgelist', file_format=file_format)
    # generate graph and edgelist dataframe, and return dataframe
    graph = None
    def generate_graph_df():
        graph = graph_generator()
        graph.df_edgelist = graph_to_dataframe(graph)
        return graph.df_edgelist

    graph_df = compute_cached_df(generate_graph_df, filename)

    # if retrieved from the cache, regenerate graph
    if graph is None:
        graph = dataframe_to_graph(graph_df)
        graph.df_edgelist = graph_df

    if not hasattr(graph, 'name'):
        graph.name = graph_name

    #print('graph_name: {}'.format(graph_name))
    #print('graph.name: {}'.format(graph.name))

    return graph


def compute_cached_reachability_labels(graph, graph_name=None, add_missing_nodes=True,
                                       datasets_dir='datasets', file_format='csv.gz'):
    graph.df_nodedata = compute_cached_reachability_labels_df(graph, graph_name=graph_name,
                                                              datasets_dir=datasets_dir, file_format=file_format)
    dataframe_to_reachability_labels(graph.df_nodedata, graph)
    if add_missing_nodes and set(graph.nodes) < set(graph.df_nodedata.index):
        graph.add_nodes_from(set(graph.df_nodedata.index) - set(graph.nodes))
    
    return graph



<u>Test</u> that `compute_cached_graph()` works

In [ ]:
expect_graph = graphs.commit_graph_Stolee()
actual_graph = compute_cached_graph(graphs.commit_graph_Stolee, 'commit_graph_Stolee-test')
assert expect_graph.nodes == actual_graph.nodes
print('ok - retrieved/computed graph has the same nodes')
assert expect_graph.edges == actual_graph.edges
print('ok - retrieved/computed graph has the same edges')
print('\npublic attributes of retrieved/computed graph')
for (attr, val) in actual_graph.__dict__.items():
    if not isinstance(val, type) and not attr.startswith('_'):
        print('- {:s} ({})'.format(attr, type(val)))
        
print('\npublic attributes of example graph')
for (attr, val) in expect_graph.__dict__.items():
    if not isinstance(val, type)  and not attr.startswith('_'):
        print('- {:s} ({})'.format(attr, type(val)))

ok - retrieved/computed graph has the same nodes
ok - retrieved/computed graph has the same edges

public attributes of retrieved/computed graph
- graph (<class 'dict'>)
- df_edgelist (<class 'pandas.core.frame.DataFrame'>)
- nodes (<class 'networkx.classes.reportviews.NodeView'>)

public attributes of example graph
- graph (<class 'dict'>)
- pos (<class 'dict'>)
- nodes (<class 'networkx.classes.reportviews.NodeView'>)


<u>Test</u> that `compute_cached_reachability_labels()` works

In [ ]:
some_graph = graphs.commit_graph_Stolee()
compute_cached_reachability_labels(some_graph, 'commit_graph_Stolee-test')
compute_cached_reachability_labels(some_graph, 'commit_graph_Stolee-test')

#print('')
print('public attributes of the graph named "{}"'.format('commit_graph_Stolee-test'))
for (attr, val) in some_graph.__dict__.items():
    if not isinstance(val, type)  and not attr.startswith('_'):
        print('- {!s} ({})'.format(attr, type(val)))

print('')
df = some_graph.df_nodedata
print('check that the dataframe has all the columns')
assert set(df.columns) == set(['level', 'f_min', 'min', 'post', 'in degree', 'out degree', 'degree'])
print('- columns: {}'.format(df.columns.tolist()))

print('check that the dataframe has all the rows')
assert set(df.index) == set(some_graph.nodes)
print('- rows:    {}...'.format(list(example_graph.nodes)[0:5]))        

print('check that graph has "lvl" attribute')
assert hasattr(some_graph, 'lvl')
print('- lvls:    {}...'.format({k: some_graph.lvl[k] for k in list(some_graph.lvl)[:5]}))

print('check that graph has "mpi_ext" attribute')
assert hasattr(some_graph, 'mpi_ext')
print('- mpi_ext: {}...'.format({k: some_graph.mpi_ext[k] for k in list(some_graph.mpi_ext)[:2]}))

some_graph.df_nodedata.head()

public attributes of the graph named "commit_graph_Stolee-test"
- graph (<class 'dict'>)
- pos (<class 'dict'>)
- df_nodedata (<class 'pandas.core.frame.DataFrame'>)
- lvl (<class 'dict'>)
- mpi_ext (<class 'dict'>)
- nodes (<class 'networkx.classes.reportviews.NodeView'>)

check that the dataframe has all the columns
- columns: ['f_min', 'min', 'post', 'level', 'in degree', 'out degree', 'degree']
check that the dataframe has all the rows
- rows:    ['A', 'a7', 'a5', 'a4', 'a3']...
check that graph has "lvl" attribute
- lvls:    {'b0': 0, 'a1': 1, 'a2': 2, 'a3': 3, 'a4': 4}...
check that graph has "mpi_ext" attribute
- mpi_ext: {'b0': {'f_min': 1, 'min': 1, 'post': 1}, 'a1': {'f_min': 1, 'min': 1, 'post': 2}}...


,f_min,min,post,level,in degree,out degree,degree
node,,,,,,,
b0,1,1,1,0,3,0,3
a1,1,1,2,1,2,1,3
a2,1,1,3,2,1,1,2
a3,1,1,4,3,2,1,3
a4,1,1,5,4,1,1,2


----

In [ ]:
#hide
# this should be the last cell of the notebook
from nbdev.export import notebook2script
notebook2script()

Converted 01_tools.ipynb.
Converted 02_related.ipynb.
Converted 03_example_graphs.ipynb.
Converted 05_reachability_index.ipynb.
Converted 06_levels.ipynb.
Converted 07_interval_labels.ipynb.
Converted 08_reach.ipynb.
Converted 09_git.ipynb.
Converted 10_checkpoint.ipynb.
Converted A.09_git_explore.ipynb.
Converted index.ipynb.
